In [1]:
import numpy as np
import pandas as pd
from statsmodels.regression.quantile_regression import QuantReg

def run_election_night_pred(Y, X, alpha, gamma, tinit=100, split_size=0.75, update_method="Simple", momentum_bw=0.95):
    T = len(Y)
    alpha_trajectory = np.empty(T - tinit)
    adapt_err_seq = np.zeros(T - tinit)
    no_adapt_err_seq = np.zeros(T - tinit)
    alphat = alpha

    for t in range(tinit, T):
        # Split data into training and calibration set
        train_points = np.random.choice(t-1, round(split_size * (t-1)), replace=False)
        cal_points = np.setdiff1d(np.arange(t-1), train_points)
        
        X_train = X[train_points, :]
        Y_train = Y[train_points]

        #print(X_train.shape, Y_train.shape)

        X_cal = X[cal_points, :]
        Y_cal = Y[cal_points]

        # Fit quantile regression on training data
        model_upper = QuantReg(Y_train, X_train).fit(q=1 - alpha / 2)
        model_lower = QuantReg(Y_train, X_train).fit(q=alpha / 2)

        # Compute conformity score on calibration set and on new data example
        pred_low_for_cal = model_lower.predict(X_cal)
        pred_up_for_cal = model_upper.predict(X_cal)
        scores = np.maximum(Y_cal - pred_up_for_cal, pred_low_for_cal - Y_cal)
        q_up = model_upper.predict(X[t])
        q_low = model_lower.predict(X[t])
        new_score = max(Y[t] - q_up, q_low - Y[t])

        # Compute errt for both methods
        conf_quant_naive = np.quantile(scores, 1 - alpha)
        no_adapt_err_seq[t - tinit] = float(conf_quant_naive < new_score)

        if alphat >= 1:
            adapt_err_seq[t - tinit] = 1
        elif alphat <= 0:
            adapt_err_seq[t - tinit] = 0
        else:
            conf_quant_adapt = np.quantile(scores, 1 - alphat)
            adapt_err_seq[t - tinit] = float(conf_quant_adapt < new_score)

        # Update alphat
        alpha_trajectory[t - tinit] = alphat
        if update_method == "Simple":
            alphat += gamma * (alpha - adapt_err_seq[t - tinit])
        elif update_method == "Momentum":
            w = np.array([momentum_bw ** i for i in range(t - tinit + 1)])
            w /= w.sum()
            alphat += gamma * (alpha - np.dot(adapt_err_seq[:t - tinit + 1], w))

        if t % 100 == 0:
            print(f"Done {t} time steps")

    return alpha_trajectory, adapt_err_seq, no_adapt_err_seq

# Example usage
Y = np.random.rand(500) # Simulated target values
X = np.random.rand(500, 5) # Simulated predictor variables
alpha = 0.05
gamma = 0.01

results = run_election_night_pred(Y, X, alpha, gamma)
alpha_trajectory, adapt_err_seq, no_adapt_err_seq = results

/var/folders/kb/88rcrqqd1_l8ftjs5bctjkqh0000gn/T/ipykernel_72268/2686134704.py:39: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  no_adapt_err_seq[t - tinit] = float(conf_quant_naive < new_score)
/var/folders/kb/88rcrqqd1_l8ftjs5bctjkqh0000gn/T/ipykernel_72268/2686134704.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  adapt_err_seq[t - tinit] = float(conf_quant_adapt < new_score)


Done 100 time steps


/Users/hanleyan/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/hanleyan/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/hanleyan/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


Done 200 time steps


/Users/hanleyan/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/hanleyan/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


Done 300 time steps


/Users/hanleyan/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/hanleyan/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


Done 400 time steps


/Users/hanleyan/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/hanleyan/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/hanleyan/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


In [37]:
print(results[1].mean())

0.055
